In [ ]:
import tensorflow as tf
import numpy as np

## Simple Math

In [ ]:
inputs = np.random.rand(100, 20)
weights = np.random.rand(20, 1)

In [ ]:
graph_1 = tf.Graph()
with graph_1.as_default():
    x = tf.placeholder(dtype=tf.float64, name='x')
    w = tf.placeholder(dtype=tf.float64, name='w')
    y = tf.matmul(x, w)

In [ ]:
# tf.global_variables_initializer()
with tf.Session(graph=graph_1) as sess:
    y_val = sess.run(y, feed_dict={x: inputs, w: weights})

In [ ]:
y_val.shape

## LSTM Trials

In [ ]:
sequence_inputs = list()
for i in range(100):
    sequence_inputs.append(np.random.rand(20, 50))
gold_value_array = np.random.rand(100, 1) * 10

In [ ]:
gold_value_array.dtype, sequence_inputs[0].dtype

In [ ]:
graph_2 = tf.Graph()

with graph_2.as_default():
    weights = tf.Variable(dtype=tf.float64, name='weights',
                          initial_value=tf.truncated_normal(shape=[128, 1], stddev=0.001, dtype=tf.float64))
    biases = tf.Variable(dtype=tf.float64, name='weights', 
                         initial_value=tf.truncated_normal(shape=[1, 1], stddev=0.001, dtype=tf.float64))
    num_lstm_units = 128
    
    inputs_series = tf.placeholder(dtype=tf.float64, name='inputs_series', shape=(100, 20, 50))
    gold_values = tf.placeholder(dtype=tf.float64, name='gold_value')
    
    init_state = tf.zeros(shape=[100], dtype=tf.float64, name='init_state')
    
    lstm_1 = tf.contrib.rnn.BasicLSTMCell(num_lstm_units)
    
    states_series, current_state = \
        tf.nn.dynamic_rnn(lstm_1, inputs_series, init_state, dtype=tf.float64)
    final_predictions = tf.matmul(current_state.h, weights) + biases
    
    cost = tf.losses.absolute_difference(final_predictions, gold_values)
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

In [ ]:
current_state.h.shape

In [ ]:
epoch_reporting_interval = 50

with tf.Session(graph=graph_2) as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(1, 1001):
        _, c = \
            sess.run([optimizer, cost], 
                feed_dict={
                    inputs_series: sequence_inputs, 
                    gold_values: gold_value_array
                }
            )
        
        if (i % epoch_reporting_interval == 0):
            print("Training epoch: ", i, ", Cost: ", c)
            
    final_predictions = \
        sess.run(
            [final_predictions], 
            feed_dict={
                inputs_series: sequence_inputs, 
                gold_values: gold_value_array
            }
        )

In [ ]:
final_predictions